In [1]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt

# Path to your DICOM file
dcm_file_path = "radioprotect/data/SAMPLE_001/DR.1.2.246.352.221.485912002656867480216350827647519239102.dcm"

# Load the DICOM file
try:
    dcm = pydicom.dcmread(dcm_file_path)
except Exception as e:
    print(f"Error loading DICOM file: {e}")
    exit()

In [5]:
# Extract pixel data as a NumPy array
if hasattr(dcm, "pixel_array"):
    image = dcm.pixel_array
    print(f"Image shape: {image.shape}")
else:
    print("No pixel data found in the DICOM file.")
    exit()

# Optional: Display the CT image
plt.imshow(image, cmap="gray")
plt.title("CT Scan")
plt.axis("off")  # Hide axes
plt.show()

# Optional: Access metadata (e.g., Patient ID, Slice Thickness)
print(f"Patient ID: {dcm.get('PatientID', 'N/A')}")
print(f"Slice Thickness: {dcm.get('SliceThickness', 'N/A')} mm")


No pixel data found in the DICOM file.


NameError: name 'image' is not defined

: 

In [2]:
dcm.keys

<bound method Dataset.keys of Dataset.file_meta -------------------------------
(0002,0000) File Meta Information Group Length  UL: 188
(0002,0001) File Meta Information Version       OB: b'\x00\x01'
(0002,0002) Media Storage SOP Class UID         UI: Deformable Spatial Registration Storage
(0002,0003) Media Storage SOP Instance UID      UI: 1.2.246.352.221.485912002656867480216350827647519239102
(0002,0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002,0012) Implementation Class UID            UI: 1.2.246.352.70.2.1.160.3
(0002,0013) Implementation Version Name         SH: 'DCIE 2.2'
-------------------------------------------------
(0008,0005) Specific Character Set              CS: 'ISO_IR 192'
(0008,0012) Instance Creation Date              DA: '20250408'
(0008,0013) Instance Creation Time              TM: '110121'
(0008,0016) SOP Class UID                       UI: Deformable Spatial Registration Storage
(0008,0018) SOP Instance UID                    UI:

In [6]:
from pathlib import Path

p = Path('radioprotect/data/SAMPLE_002')

contents = list(p.iterdir())

In [7]:
for c in contents:
    if "dcm" not in c.name:
        print(c.name)